In [ ]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_random_exponential
import os

import openai

from utils.embedding_utils import get_embedding, sliding_window

In [ ]:
victim = "Oppie"

df = pd.read_csv(f"../data/{victim}/db/df_text.csv")
df.head()

In [ ]:
df_filtered = df

In [8]:
window_size = 256  # Length of text chunks
stride = 192 # Stride of sliding window; have a bit of overlap

text_chunks = []
embeddings = []

for i in tqdm(range(len(df_filtered))):
    text = df_filtered['text'].values[i].replace('\n', ' ').strip()
    text_chunks_i = list(sliding_window(text, window_size, stride))
    if len(text_chunks_i) > 80:  # If text too long, truncate
        text_chunks_i = text_chunks_i[:80]
    embeddings_i = [get_embedding(text) for text in text_chunks_i]
    text_chunks += text_chunks_i
    embeddings += embeddings_i

  0%|          | 0/1 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
embeddings = np.array(embeddings, dtype=np.float64)

In [ ]:
data = [text_chunks]

# Transpose the data to have the inner lists as rows
transposed_data = list(map(list, zip(*data)))

# Column names for the DataFrame
columns = ['text_chunks']

# Create the DataFrame
df = pd.DataFrame(transposed_data, columns=columns)

In [ ]:
df.to_csv(f'../data/{victim}/db/text_chunks.csv', index=False)
np.save(f'../data/{victim}/db/embeddings.npy', embeddings)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from utils.embedding_utils import get_embedding

def semantic_search(query_embedding, embeddings):
    """Load context prompt."""
    similarities = cosine_similarity([query_embedding], embeddings)[0]
    ranked_indices = np.argsort(-similarities)
    return ranked_indices

In [ ]:
emb = get_embedding("Tell me about the pion")
semantic_search(emb, embeddings)

In [ ]:
text_chunks[semantic_search(emb, embeddings)[0]]